In [ ]:
import matplotlib.pyplot as plt

# Assuming you have a list to store losses
train_losses = []

# Update your training loop to record loss
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, masks in train_loader:
        optimizer.zero_grad()
        outputs = model(images.unsqueeze(1).float())
        loss = criterion(outputs, masks.unsqueeze(1).float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    train_losses.append(epoch_loss)  # Store the average loss
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}')

# Plot the training loss
plt.plot(train_losses, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.show()


In [ ]:
def visualize_predictions(model, val_loader):
    model.eval()
    with torch.no_grad():
        for images, masks in val_loader:
            outputs = model(images.unsqueeze(1).float())
            preds = torch.sigmoid(outputs) > 0.5  # Apply threshold

            # Display original image, mask, and prediction
            for i in range(len(images)):
                plt.figure(figsize=(12, 4))

                plt.subplot(1, 3, 1)
                plt.title('Original Image')
                plt.imshow(images[i].cpu().numpy(), cmap='gray')
                plt.axis('off')

                plt.subplot(1, 3, 2)
                plt.title('Ground Truth Mask')
                plt.imshow(masks[i].cpu().numpy(), cmap='gray')
                plt.axis('off')

                plt.subplot(1, 3, 3)
                plt.title('Predicted Mask')
                plt.imshow(preds[i].cpu().numpy(), cmap='gray')
                plt.axis('off')

                plt.show()
            break  # Visualize only the first batch


In [ ]:
# Save the model
torch.save(model.state_dict(), 'unet_model.pth')


In [ ]:
pip install fastapi uvicorn


In [ ]:
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
import numpy as np
import cv2
import torch

# Load the model
model = UNet(in_channels=1, out_channels=1)
model.load_state_dict(torch.load('unet_model.pth'))
model.eval()

app = FastAPI()

def preprocess_image(image):
    # Preprocess the uploaded image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = apply_clahe(image)
    image = normalize_image(image)
    return image

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    # Read the uploaded image
    contents = await file.read()
    np_array = np.frombuffer(contents, np.uint8)
    image = cv2.imdecode(np_array, cv2.IMREAD_COLOR)

    # Preprocess the image
    processed_image = preprocess_image(image)
    input_tensor = torch.tensor(processed_image).unsqueeze(0).unsqueeze(0).float()  # Add batch and channel dimensions

    # Perform inference
    with torch.no_grad():
        output = model(input_tensor)
        preds = torch.sigmoid(output) > 0.5  # Apply threshold

    # Convert prediction to numpy array for response
    pred_mask = preds.squeeze().numpy()
    return JSONResponse(content={"mask": pred_mask.tolist()})  # Return the mask as a list

# Run the application using: uvicorn main:app --reload


In [ ]:
uvicorn main:app --reload
